## FluSight Forecast Usage and Examples
#### Andrew Attilio
#### 2024-11-16
Note that most of the code in this notebook is not runnable, and is just provided as a visual example. 

### Quick Facts
The Flusight Forecast pipeline is partially constructed in:
- `src/scripts/flusight_forecast.py

And an example of a now-deprecated,
 pipeline can be found in:
- `src/scripts/forecast_all_states_deprecated.py

To generate 52 shell scripts for the 52 locations
(where each script would trigger a flusight pipeline for that location)
see:
- `src/scripts/generate_shell_scripts.py

For quick access to relevant directory paths, see the following example:

In [6]:
# Example for accessing directory paths
from src.utils import paths

print(paths.DATASETS_DIR)
print(paths.OUTPUT_DIR)
print(paths.HOSP_FORECAST_DIR)

/Users/macbook/projects/pmcmc_forecast/datasets
/Users/macbook/projects/pmcmc_forecast/output
/Users/macbook/projects/pmcmc_forecast/src/hosp_forecast


## Details for Each Component
The functionality is split into 3 main components:
- PMCMC 
- Trend Forecast
- Hospital Forecast

Let's explore one component at a time. 


### PMCMC 
The PMCMC module (see `src/pmcmc/`) allows us to take in CDC hospitalization data and estimate a transmission rate $\beta$.
We have a Particle Filter (PF) wrapped in an MCMC loop, where the MCMC proposes parameters for the PF.

#### Config File
Most settings for the PMCMC loop are configurable in `src/pmcmc/config.toml`.  

#### Particle Filter
The PF functionality is contained within a ParticleCloud class (see `/src/pmcmc/particle_cloud.py`). 
This ParticleCloud is wrapped in a PaticleFilterAlgo class (see `/src/pmcmc/filter_algo.py`).
The ParticleFilterAlgo has a `run()` method that is triggered within the MCMC algorithm below. 

#### MCMC
...

In [ ]:
# Example call for main pmcmc algo. 
from src.pmcmc import main as pmcmc

location_code = '04'  # Arizona location code
target_date = '2023-10-28'
beta_estimates = pmcmc.main(location_code, target_date)

These $\beta$ estimates can then be passed into our Trend Forecasting algorithm. 

### Trend Forecasting


Note that the Trend Forecasting algorithm relies on the use of covariate data. Currently, we are grabbing weather data from OpenMeteoAPI (open-meteo.com) and Google Search Trends. 

We keep a cache/database of covariate data in `/datasets/covariates/database` to avoid grabbing the same data more than once, and to avoid rate limits. See `src/trend_forecast/update_covariates.py` for the functinoality of grabbing this covariate data, especially the `update_all_covariate_data()` function. 

Currently, the trend_forecast.R script is very inflexible, and expects a precise format for the CSV file that it reads data from.  

The Trend Forecast algorithm can be run as follows, where the R script is wrapped in a Python function in `src/trend_forecast/main.py`:

In [ ]:
# Example call for main trend forecast algo.
from src.trend_forecast.main import main as tf

beta_forecasts = tf(beta_estimates, location_code, target_date)

The Python wrapper runs an R subprocess, passing in file paths and other information as arguments to the R subprocess.  

Now, these $\beta$ forecasts can be passed into our hospital forecast algorithm to solve the SIRH system and produce hospitalization forecasts.

### Hospital Forecasting
The hospital forecast takes in the beta estimates from Trend Forecasting
and outputs a CSV file with hospitalization estimates for 4 weeks into the future.  


In [ ]:
from src.hosp_forecast.main import main as hf

# This function call will output the hospital estimates
# to a csv file. 
hf(beta_estimates, location_code, target_date)

## Potential Issues / Improvements

- Currently, the MCMC prior is hard-coded as uniform. See the `get_prior()` function in `src/pmcmc/main.py`.
- MCMC only currently accomodates 1 parameter. The `cov_update` function is not yet implemented, which would update a covariance matrix for multiple parameters. 
- We only get a few-hundred MCMC iterations in any reasonable amount of time. So, I have not been using the burn-in setting in `config.toml` for anything yet. 
- The final output format needs to be fixed up according to FluSight guidelines.
- Probably lots of other little things. 
- The documentation in `/docs/` was started a few months ago. It is not up to date, and there is not much information there. 
